In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'tube_outside_diameter_metric'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
field='TubeO_D_'
fields = [field]
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

3124


In [5]:
# print(len(df[(df['OutsideDiameter'].astype(str)!='nan')&(df['mm'].astype(str)!='[]')]))
# df[(df['OutsideDiameter'].astype(str)!='nan')&(df['mm'].astype(str)!='[]')]
df.head(0)

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, TubeO_D_]
Index: []

In [6]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [18]:
df[0:500]

buckets external_id  \
0                       Pipes & Tubing      405598   
1                       Pipes & Tubing       54705   
2                       Pipes & Tubing       54706   
3                       Pipes & Tubing       54707   
4                       Pipes & Tubing       54708   
5                       Pipes & Tubing       54709   
6                       Pipes & Tubing       54702   
7                       Pipes & Tubing       54703   
8                       Pipes & Tubing       54701   
9                       Pipes & Tubing       54704   
10                      Pipes & Tubing       54700   
11                      Pipes & Tubing  NT10004-5C   
12                      Pipes & Tubing   NT10004-M   
13                      Pipes & Tubing  NT10004RD5   
14                      Pipes & Tubing  NT10004YW5   
15                      Pipes & Tubing  NT10006-5C   
16                      Pipes & Tubing  NT10008-5C   
17                      Pipes & Tubing     NT10008   
18                      Pipes & Tubing     NT10006   
19                      Pipes & Tubing     TP16004   
20                      Pipes & Tubing     NT10004   
21                      Pipes & Tubing     NT10010   
22                      Pipes & Tubing   NT10006BU   
23                      Pipes & Tubing   NT10008RD   
24                      Pipes & Tubing   NT10006RD   
25                      Pipes & Tubing     NT10012   
26                      Pipes & Tubing   NT10006OR   
27                      Pipes & Tubing   NT10006YW   
28                      Pipes & Tubing   NT10006GN   
29                      Pipes & Tubing   NT10008BU   
30                      Pipes & Tubing     NT10002   
31                      Pipes & Tubing    NT100025   
32                      Pipes & Tubing   NT10004RD   
33                      Pipes & Tubing   NT10008OR   
34                      Pipes & Tubing   NT10004BU   
35                      Pipes & Tubing     NT10003   
36                      Pipes & Tubing   NT10004YW   
37                      Pipes & Tubing   NT10008GN   
38                      Pipes & Tubing     NT10005   
39                      Pipes & Tubing   NT10004GN   
40                      Pipes & Tubing   NT10004OR   
41                      Pipes & Tubing  NT10006BU5   
42                      Pipes & Tubing  NT100025RD   
43                      Pipes & Tubing   NT10008YW   
44                      Pipes & Tubing  NT10006RD5   
45                      Pipes & Tubing   NT10002BU   
46                      Pipes & Tubing  NT100025BU   
47                      Pipes & Tubing  NT100025GN   
48                      Pipes & Tubing   NT10002RD   
49                      Pipes & Tubing   NT10002GN   
50                      Pipes & Tubing   NT10003RD   
51                      Pipes & Tubing   NT10003BU   
52                      Pipes & Tubing   NT10003GN   
53                      Pipes & Tubing  PU34004BK5   
54                      Pipes & Tubing     PU34008   
55                      Pipes & Tubing     PU34006   
56                      Pipes & Tubing     PU34004   
57                      Pipes & Tubing       54305   
58                      Pipes & Tubing   PU34006BK   
59                      Pipes & Tubing   PU34008BK   
60                      Pipes & Tubing   PU34004BK   
61                      Pipes & Tubing   PU34004BU   
62                      Pipes & Tubing     PU34005   
63                      Pipes & Tubing       54306   
64                      Pipes & Tubing   PU34006BU   
65                      Pipes & Tubing   PU34005BK   
66                      Pipes & Tubing    PU34004R   
67                      Pipes & Tubing     PU34025   
68                      Pipes & Tubing   PU34025BK   
69                      Pipes & Tubing     PU34002   
70                      Pipes & Tubing    PU34006R   
71                      Pipes & Tubing   PU34002BK   
72                      Pipes & Tubing     PT23004   
73                      Pipes & Tub

In [17]:
# start=r'(?i)(^.{0,10})|()'
# df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(?i)(\d+.?mm(?:\sx?\s|\s?\-\s?)\d*.?m?m?|\d+.?mm)|()''' 
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))



pat=r'(?i)(\d+.?mm(?:\sx?\s|\s?\-\s?)\d*.?m?m?|\d+.?mm)|()'
df['matches']=df[field].apply(lambda x: re_extract(pat, str(x)))

pat=r'(?i)(mm)|()'
df['mm']=df[field].apply(lambda x: re_extract(pat, str(x)))


custom=df[(df[field].astype(str)!='nan')&((df['mm'].astype(str)!='[]'))]
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x)))#.apply(lambda x: re.sub(r'(?i)\s?mm\s?',' mm',str(x))).apply(lambda x: re.sub(r'\s?"\s?',' in',str(x))).apply(lambda x: f'{x} in').apply(lambda x: re.sub(r'\s?in\s?in',' in',str(x)))                                          
# custom[curation_col]=custom[field].apply(lambda x: f'{x}"').apply(lambda x: re.sub(r'""','"',str(x)))
print(len(custom))
matchcustom=custom[['external_id',curation_col]]
matchcustom[curation_col]=matchcustom[curation_col].apply(lambda x: re.sub(r'(?i)\s?mm',' mm',str(x))).apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x)))
matchcustom=matchcustom[matchcustom[curation_col].astype(str)!='']
custom[field].explode().value_counts()

0


Series([], Name: TubeO_D_, dtype: int64)

In [9]:
not_match=df[(df[field].astype(str)=='nan')&(df['mm'].astype(str)=='[]')]
print(len(not_match[(not_match['ld'].astype(str)!='[]')|(not_match['name'].astype(str)!='[]')]))
print(len(not_match[(not_match['ld'].astype(str)=='[]')&(not_match['name'].astype(str)=='[]')]))
not_match[(not_match['ld'].astype(str)=='[]')&(not_match['name'].astype(str)=='[]')][0:500]

0
0


Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, TubeO_D_, name, ld, matches, mm]
Index: []

In [10]:
# df[(df['name'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')][0:500]

In [11]:
# two=df[(df['matches'].astype(str)=='[]')]
# print(len(two[two['name'].astype(str)!='[]']))
# two=two[two['name'].astype(str)!='[]']
# two[curation_col]=two['name'].apply(lambda x:re.sub(r'(?i)P\.?S\.?I\.?','',str(x))).apply(lambda x: re.sub(r'.+\-\s?',"['",str(x))).apply(lambda x: re.sub(r'(?<!\,)000',',000',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"\s?'\]",'"]',str(x))).apply(lambda x: re.sub(r"\s?\s?\s?'\s?\,\s?'",'","',str(x))).apply(lambda x: re.sub(r'200"\,"125','125","200',str(x))).apply(lambda x: re.sub(r'\["1\,500"\,"100"\,"2,000 "\]','["100","1,500","2,000"]',str(x))).apply(lambda x: re.sub(r'\s?"\]','"]',str(x)))                                                    
# two[curation_col].explode().value_counts()
# matchtwo=two[['external_id',curation_col]]

In [12]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [19]:
na=df[(df['name'].astype(str)=='[]')&(df['ld'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
# print(len(na)+len(matchcustom)+len(matchtwo))

3115


In [24]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchtwo))

# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [11]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [25]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [20]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [47]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchtwo) 